# Limpieza de datos con PySpark: Data Science Job Posting on Glassdoor

Los [datos](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ETYTQ0c-i6FLjM8rZ4iT1cgB6ipFAkainM-4V9M8DXsBiA?e=PeMtvh) fueron extraídos (scrapeados) del sitio web de Glassdoor y recoge los salarios de distintos puestos relacionados a Data.

In [0]:
from pyspark.sql import functions as F

df = spark.read.option("header", "true") \
               .option("delimiter", ";") \
               .option("multiline", "true") \
               .option("quote", "\"") \
               .option("escape", "\"") \
               .csv("/FileStore/Examen/ds_jobs.csv")
# Mostrar los datos bien estructurados
display(df)

### Resolver los siguientes requerimientos, para cada operación/moficación imprima como van quedadndo los cambios.

1. Cargar los datos y mostrar el esquema o la informacion de las columnas y el tip de dato de cada columna

In [0]:
# Mostrar el esquema detallado
print("Esquema del DataFrame:")
df.printSchema()

# Mostrar información de las columnas y tipos de datos
print("Columnas y tipos de datos:")
for field in df.schema.fields:
    print(f"{field.name}: {field.dataType}")

2. Eliminar duplicados

In [0]:
# Eliminar duplicados basados en todas las columnas
df_no_duplicates = df.dropDuplicates()

# Calcular los valores eliminados
df_duplicates_removed = df.subtract(df_no_duplicates)

# Mostrar los valores eliminados
print(f"Total de filas eliminadas: {df_duplicates_removed.count()}")
display(df_duplicates_removed)

# Mostrar los datos sin duplicados
display(df_no_duplicates)

3. Decidir que hacer con los datos faltantes 

Lo mejor seria eliminar esos datos ya que perderia el valor al completo si nos inventamos los datos

4. Decidir que hacer con los valores nulos


Al no tener muchos valores nulos lo mejor seria rellenarlos con datos promedios de la empresa segun antigüedad habria que juzgar cada dato para darle un valor apropiado

In [0]:
from pyspark.sql.functions import col
from functools import reduce

# Filtrar filas con valores nulos en cualquier columna
df_null_values = df_no_duplicates.filter(
    reduce(
        lambda x, y: x | y,
        [col(c).isNull() for c in df_no_duplicates.columns]
    )
)

# Mostrar el número de filas con valores nulos
print(f"Total de filas con valores nulos: {df_null_values.count()}")

# Mostrar las filas con valores nulos
display(df_null_values)


5. ¿Cuántos registros tiene el csv?



In [0]:
# Contar el número de registros
total_registros = df_no_duplicates.count()

# Mostrar el resultado
print(f"Total de registros en el CSV: {total_registros}")

6. Mostrar los valores únicos de `Job title` 

In [0]:
# Obtener los valores únicos de la columna Job Title
unique_job_titles = df.select("Job Title").distinct()

# Mostrar los valores únicos
print(f"Total de valores únicos en 'Job Title': {unique_job_titles.count()}")
display(unique_job_titles)

7. Remover la letra `K` de la columna `Salary Estimate` y multiplicar por 1000.

In [0]:
from pyspark.sql.functions import regexp_replace, split, col

# Limpiar la columna Salary Estimate (eliminar '$' y 'K', multiplicar por 1000)
df_transformed = df.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), r"\$", "")  # Eliminar el símbolo '$'
)

df_transformed = df_transformed.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), r"K", "")  # Eliminar la letra 'K'
)

# Extraer el salario mínimo y máximo y multiplicar por 1000 si es necesario
df_transformed = df_transformed.withColumn(
    "Salary Min",
    (split(col("Salary Estimate"), "-")[0].cast("double") * 1000)  # Multiplicar por 1000
).withColumn(
    "Salary Max",
    (split(col("Salary Estimate"), "-")[1].cast("double") * 1000)  # Multiplicar por 1000
)

# Mostrar el DataFrame con las nuevas columnas Salary Min y Salary Max
display(df_transformed.select("Salary Estimate"))



8. Mostrar los valores únicos del campo `Salary Estimate`

In [0]:
# Obtener los valores únicos de la columna Salary Estimate
unique_salary_estimates = df.select("Salary Estimate").distinct()

# Contar el número de valores únicos
total_unique = unique_salary_estimates.count()
print(f"Total de valores únicos en 'Salary Estimate': {total_unique}")

# Mostrar los valores únicos
display(unique_salary_estimates)

9. Eliminar `(Glassdoor est.)` y `(Employer est.)` del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import regexp_replace, trim

# Eliminar "(Glassdoor est.)" y "(Employer est.)" del campo Salary Estimate
df_cleaned = df.withColumn(
    "Salary Estimate",
    trim(regexp_replace(col("Salary Estimate"), r"\s*\(Glassdoor est\.|\(Employer est\.\)", ""))
)

# Mostrar los datos limpios
display(df_cleaned)


10. Mostrar de mayor a menor los valores del campo `Salary Estimate`

In [0]:
from pyspark.sql.functions import regexp_extract, col, asc, desc

# Extraer el valor máximo del rango salarial y convertirlo a numérico
df_with_salary_numeric = df.withColumn(
    "Salary Max",
    (regexp_extract(col("Salary Estimate"), r"-\$(\d+)K", 1).cast("double") * 1000)
)

# Ordenar por el valor máximo del salario en orden descendente
df_sorted = df_with_salary_numeric.orderBy(col("Salary Max").desc())

# Mostrar los datos ordenados
display(df_sorted.select("Salary Estimate", "Salary Max"))


11. De la columna `Job Description` quitar los saltos de linea `\n` del texto

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Remover los saltos de línea (\n) de la columna Job Description
df_cleaned = df.withColumn(
    "Job Description",
    regexp_replace(col("Job Description"), r"[\r\n]+", " ")
)

# Mostrar el DataFrame limpio
display(df_cleaned.select("Job Description"))


12. De la columna `Rating` muestre los valores unicos.

In [0]:
# Obtener los valores únicos de la columna Rating
unique_ratings = df.select("Rating").distinct()

# Contar el número de valores únicos
total_unique = unique_ratings.count()
print(f"Total de valores únicos en 'Rating': {total_unique}")

# Mostrar los valores únicos
display(unique_ratings)


13. Del campo `Rating` reemplazar los `-1.0` por `0.0`.

In [0]:
from pyspark.sql.functions import when, col

# Reemplazar los valores -1.0 por 0.0 en la columna Rating
df_updated = df.withColumn(
    "Rating",
    when(col("Rating") == -1.0, 0.0).otherwise(col("Rating"))
)

# Mostrar el DataFrame actualizado
display(df_updated.select("Rating"))


14. Mostrar los valores unicos y ordenar los valores del campo `Company Name`.

In [0]:
from pyspark.sql.functions import col

# Obtener los valores únicos y ordenarlos
unique_company_names = df.select("Company Name").distinct().orderBy(col("Company Name").asc())

# Contar el número de valores únicos
total_unique = unique_company_names.count()
print(f"Total de valores únicos en 'Company Name': {total_unique}")

# Mostrar los valores únicos y ordenados
display(unique_company_names)


15. Quitar todos los caracteres innecesarios que encuentres en el campo `Company Name`. Por ejemplo los saltos de linea `\n`

In [0]:
from pyspark.sql.functions import regexp_replace, trim, col

# Limpiar la columna Company Name
df_cleaned = df.withColumn(
    "Company Name",
    trim(regexp_replace(col("Company Name"), r"[\r\n\t]", ""))  # Elimina \r, \n y \t
)

# Mostrar los datos limpios
display(df_cleaned.select("Company Name"))


16. En el campo `Location` convertir esa columna en dos: `City` y `State`. Las ciudades que tengas en `Location` asignar a la columna `City`. Lo mismo para `State`. Luego elimine la columna `Location`.

In [0]:
from pyspark.sql.functions import split, col

# Dividir la columna Location en City y State
df_transformed = df.withColumn("City", split(col("Location"), ", ")[0]) \
                   .withColumn("State", split(col("Location"), ", ")[1]) \
                   .drop("Location")  # Eliminar la columna original Location

# Mostrar el DataFrame con las nuevas columnas
display(df_transformed.select("City", "State"))


17. Repetir la misma lógica de la pregunta 16 pero para el campo `Headquarters`. En Headquarters dejar solo la ciudad, mientras que para el estado añadirla a una columna nueva ` Headquarter State`.

In [0]:
from pyspark.sql.functions import split, col

# Dividir la columna Headquarters en City y Headquarter State
df_transformed = df.withColumn("City", split(col("Headquarters"), ", ")[0]) \
                   .withColumn("Headquarter State", split(col("Headquarters"), ", ")[1]) \
                   .drop("Headquarters")  # Eliminar la columna original Headquarters

# Mostrar el DataFrame con las nuevas columnas
display(df_transformed.select("City", "Headquarter State"))


18. Muestre los valores únicos del campo `Headquarter State` 

In [0]:
from pyspark.sql.functions import split, col

# Dividir la columna Headquarters en City y Headquarter State
df_transformed = df.withColumn("City", split(col("Headquarters"), ", ")[0]) \
                   .withColumn("Headquarter State", split(col("Headquarters"), ", ")[1]) \
                   .drop("Headquarters")  # Eliminar la columna original Headquarters

# Obtener los valores únicos de la columna Headquarter State
unique_headquarter_states = df_transformed.select("Headquarter State").distinct()

# Contar el número de valores únicos
total_unique = unique_headquarter_states.count()
print(f"Total de valores únicos en 'Headquarter State': {total_unique}")

# Mostrar los valores únicos
display(unique_headquarter_states.orderBy(col("Headquarter State").asc()))



19. Mostrar valores unicos del campo `Size`.

In [0]:
from pyspark.sql.functions import col

# Obtener los valores únicos de la columna Size
unique_sizes = df.select("Size").distinct()

# Contar el número de valores únicos
total_unique = unique_sizes.count()
print(f"Total de valores únicos en 'Size': {total_unique}")

# Mostrar los valores únicos ordenados alfabéticamente
display(unique_sizes.orderBy(col("Size").asc()))


20. Quitar 'employee' de los registros del campo `Size`. Elimine tambien otros caracteres basura.

In [0]:
from pyspark.sql.functions import regexp_replace, trim, col

# Limpiar el campo Size eliminando 'employee' y otros caracteres basura
df_cleaned = df.withColumn(
    "Size",
    trim(
        regexp_replace(
            col("Size"), r"(?i)employee|[^a-zA-Z0-9\s\-\>]+", ""
        )
    )
)

# Mostrar el DataFrame con los valores limpios
display(df_cleaned.select("Size"))


21. Reemplazar la palabra 'to' por '-' en todos los registros del campo `Size`. Reemplazar tambien '-1' por 'Unknown'. 

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar 'to' por '-' y '-1' por 'Unknown' en el campo Size
df_transformed = df_cleaned.withColumn(
    "Size",
    regexp_replace(
        regexp_replace(col("Size"), r"\bto\b", "-"),  # Reemplazar 'to' por '-'
        r"-1", "Unknown"  # Reemplazar '-1' por 'Unknown'
    )
)

# Mostrar el DataFrame transformado
display(df_transformed.select("Size"))


22. Mostrar el tipo de dato del campo `Type of ownership` y sus registros unicos.

In [0]:
from pyspark.sql.functions import col

# Mostrar el tipo de dato de la columna
type_of_ownership_dtype = df.schema["Type of ownership"].dataType
print(f"Tipo de dato del campo 'Type of ownership': {type_of_ownership_dtype}")

# Obtener los registros únicos de la columna
unique_type_of_ownership = df.select("Type of ownership").distinct()

# Contar el número de registros únicos
total_unique = unique_type_of_ownership.count()
print(f"Total de registros únicos en 'Type of ownership': {total_unique}")

# Mostrar los registros únicos
display(unique_type_of_ownership.orderBy(col("Type of ownership").asc()))


23. Cambiar '-1' por 'Unknown' en todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import when, col

# Reemplazar '-1' por 'Unknown' en la columna Type of ownership
df_updated = df.withColumn(
    "Type of ownership",
    when(col("Type of ownership") == "-1", "Unknown").otherwise(col("Type of ownership"))
)

# Mostrar el DataFrame actualizado
display(df_updated.select("Type of ownership"))


24. Cambiar:  
-  `Company - Public` por `Public Company`  
-  `Company - Private` por `Private Company`  
-  `Private Practice / Firm` por `Private Company`  
-  `Subsidiary or Business Segment` por `Business`  
-  `College / University` por `Education`  
En todos los registros del campo `Type of ownership`.

In [0]:
from pyspark.sql.functions import when, col

# Realizar los reemplazos en la columna Type of ownership
df_updated = df.withColumn(
    "Type of ownership",
    when(col("Type of ownership") == "Company - Public", "Public Company")
    .when(col("Type of ownership") == "Company - Private", "Private Company")
    .when(col("Type of ownership") == "Private Practice / Firm", "Private Company")
    .when(col("Type of ownership") == "Subsidiary or Business Segment", "Business")
    .when(col("Type of ownership") == "College / University", "Education")
    .otherwise(col("Type of ownership"))  # Mantener los demás valores igual
)

# Mostrar el DataFrame con los valores actualizados
display(df_updated.select("Type of ownership"))


25. Mostrar el tipo de dato y los valores unicos del campo `Industry`.

In [0]:
from pyspark.sql.functions import col

# Mostrar el tipo de dato de la columna Industry
industry_dtype = df.schema["Industry"].dataType
print(f"Tipo de dato del campo 'Industry': {industry_dtype}")

# Obtener los valores únicos de la columna Industry
unique_industries = df.select("Industry").distinct()

# Contar el número de valores únicos
total_unique = unique_industries.count()
print(f"Total de valores únicos en 'Industry': {total_unique}")

# Mostrar los valores únicos
display(unique_industries.orderBy(col("Industry").asc()))


26. En el mismo campo de `Industry` reemplazar '-1' por 'Not Available' y '&' por 'and'.  Vuelva a imprimir los valores unicos en orden alfabético.

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna Industry
df_updated = df.withColumn(
    "Industry",
    regexp_replace(
        regexp_replace(col("Industry"), r"-1", "Not Available"),  # Reemplazar '-1' por 'Not Available'
        r"&", "and"  # Reemplazar '&' por 'and'
    )
)

# Obtener los valores únicos de la columna Industry, ordenados alfabéticamente
unique_industries = df_updated.select("Industry").distinct().orderBy(col("Industry").asc())

# Contar el número de valores únicos
total_unique = unique_industries.count()
print(f"Total de valores únicos en 'Industry': {total_unique}")

# Mostrar los valores únicos
display(unique_industries)


27. Para el campo `Sector`, muestre el tipo de dato y los valores únicos.

In [0]:
from pyspark.sql.functions import col

# Mostrar el tipo de dato de la columna Sector
sector_dtype = df.schema["Sector"].dataType
print(f"Tipo de dato del campo 'Sector': {sector_dtype}")

# Obtener los valores únicos de la columna Sector
unique_sectors = df.select("Sector").distinct()

# Contar el número de valores únicos
total_unique = unique_sectors.count()
print(f"Total de valores únicos en 'Sector': {total_unique}")

# Mostrar los valores únicos
display(unique_sectors.orderBy(col("Sector").asc()))


28. Aplica la misma lógica de la pregunta 26 pero sobre el campo `Sector`.

In [0]:
from pyspark.sql.functions import when, regexp_replace, col

# Reemplazar '-1' por 'Not Available' y '&' por 'and' en la columna Sector
df_transformed = df.withColumn(
    "Sector",
    regexp_replace(
        regexp_replace(col("Sector"), r"-1", "Not Available"),  # Reemplazar '-1' por 'Not Available'
        r"&", "and"  # Reemplazar '&' por 'and'
    )
)

# Obtener los valores únicos de la columna Sector, ordenados alfabéticamente
unique_sectors = df_transformed.select("Sector").distinct().orderBy(col("Sector").asc())

# Contar el número de valores únicos
total_unique = unique_sectors.count()
print(f"Total de valores únicos en 'Sector': {total_unique}")

# Mostrar los valores únicos
display(unique_sectors)


29. Para el campo `Revenue`, muestre el tipo de dato y los valores únicos en orden ascedente.

In [0]:
from pyspark.sql.functions import col

# Mostrar el tipo de dato de la columna Revenue
revenue_dtype = df.schema["Revenue"].dataType
print(f"Tipo de dato del campo 'Revenue': {revenue_dtype}")

# Obtener los valores únicos de la columna Revenue
unique_revenues = df.select("Revenue").distinct().orderBy(col("Revenue").asc())

# Contar el número de valores únicos
total_unique = unique_revenues.count()
print(f"Total de valores únicos en 'Revenue': {total_unique}")

# Mostrar los valores únicos
display(unique_revenues)


30. En el campo `Revenue`, cambiar:  
-  `-1` por `N/A`  
-  `Unknown / Non-Applicable` por `N/A`  
-  `Less than $1 million (USD)` por `Less than 1`
-  Quitar `$` y `(USD)`

In [0]:
from pyspark.sql.functions import regexp_replace, col

# Realizar los reemplazos y limpiar la columna Revenue
df_updated = df.withColumn(
    "Revenue",
    regexp_replace(
        regexp_replace(
            regexp_replace(
                regexp_replace(col("Revenue"), r"-1", "N/A"),  # Reemplazar '-1' por 'N/A'
                r"Unknown / Non-Applicable", "N/A"  # Reemplazar 'Unknown / Non-Applicable' por 'N/A'
            ),
            r"Less than \$1 million \(USD\)", "Less than 1"  # Reemplazar 'Less than $1 million (USD)' por 'Less than 1'
        ),
        r"\$|\(USD\)", ""  # Quitar los símbolos '$' y '(USD)'
    )
)

# Mostrar el DataFrame actualizado
display(df_updated.select("Revenue"))


31. Borrar el campo `Competitors`.

In [0]:
# Eliminar la columna Competitors
df_without_competitors = df.drop("Competitors")

# Mostrar el DataFrame sin la columna Competitors
display(df_without_competitors)


32. Crear tres columnas: `min_salary` (salario mínimo), `max_salary` (salario maximo) y `avg_salary` (salario promedio) a partir de los datos del campo `Salary Estimate`.

In [0]:
from pyspark.sql.functions import regexp_replace, split, col

# Paso 1: Eliminar el símbolo '$', el texto dentro de paréntesis y la letra 'K'
df_cleaned = df.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), r"\$", "")  # Eliminar '$'
)

df_cleaned = df_cleaned.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), r"\(.*\)", "")  # Eliminar paréntesis y texto dentro
)

df_cleaned = df_cleaned.withColumn(
    "Salary Estimate",
    regexp_replace(col("Salary Estimate"), r"K", "")  # Eliminar 'K'
)

# Paso 2: Dividir la columna Salary Estimate en salario mínimo y máximo
df_transformed = df_cleaned.withColumn(
    "min_salary",
    split(col("Salary Estimate"), "-")[0].cast("double")  # Obtener el salario mínimo (antes del '-')
).withColumn(
    "max_salary",
    split(col("Salary Estimate"), "-")[1].cast("double")  # Obtener el salario máximo (después del '-')
)

# Paso 3: Crear la columna avg_salary como el promedio entre min_salary y max_salary
df_transformed = df_transformed.withColumn(
    "avg_salary",
    (col("min_salary") + col("max_salary")) / 2
)

# Mostrar el DataFrame con las nuevas columnas
display(df_transformed.select("Salary Estimate", "min_salary", "max_salary", "avg_salary"))



33. Mostrar los valores unicos del campo `Founded` y el tipo de dato.

In [0]:
from pyspark.sql.functions import col

# Mostrar el tipo de dato de la columna Founded
founded_dtype = df.schema["Founded"].dataType
print(f"Tipo de dato del campo 'Founded': {founded_dtype}")

# Obtener los valores únicos de la columna Founded, ordenados ascendentemente
unique_founded = df.select("Founded").distinct().orderBy(col("Founded").asc())

# Contar el número de valores únicos
total_unique = unique_founded.count()
print(f"Total de valores únicos en 'Founded': {total_unique}")

# Mostrar los valores únicos
display(unique_founded)


34. Reemplazar '-1' por '2024' en todos los registros del campo `Founded`.

In [0]:
from pyspark.sql.functions import when, col

# Reemplazar '-1' por '2024' en la columna Founded
df_updated = df.withColumn(
    "Founded",
    when(col("Founded") == -1, 2024).otherwise(col("Founded"))  # Reemplazar '-1' por 2024
)

# Mostrar el DataFrame actualizado
display(df_updated.select("Founded"))


35. Crear una nueva columna o campo que se llame `company_age` con los datos que se deducen del campo `Founded`.

In [0]:
from pyspark.sql.functions import current_date, year, col

# Obtener el año actual
current_year = year(current_date())

# Crear una nueva columna company_age basada en la columna Founded
df_transformed = df.withColumn(
    "company_age",
    current_year - col("Founded")  # Restar el año actual con el valor de Founded
)

# Mostrar el DataFrame con la nueva columna company_age
display(df_transformed.select("Founded", "company_age"))


36. Crear una columna o campo que se llame: `Job Type` y en cada registro debe ir Senior, Junior o NA según los datos del campo `Job Title`.  
- Cambiar 'sr' o 'senior' o 'lead' o 'principal' por `Senior` en el campo `Job Type`. No olvidar las mayúsculas.
- Cambiar 'jr' o 'jr.' o cualquier otra variante por `Junior`.  
- En cualquier otro caso distinto a los anteriores añadir NA.

In [0]:
from pyspark.sql.functions import when, col

# Crear la columna Job Type basada en las condiciones del campo Job Title
df_transformed = df.withColumn(
    "Job Type",
    when(col("Job Title").rlike("(?i)sr|senior|lead|principal"), "Senior")  # Coincidir con 'sr', 'senior', 'lead', 'principal'
    .when(col("Job Title").rlike("(?i)jr|jr\\."), "Junior")  # Coincidir con 'jr', 'jr.', y otras variantes
    .otherwise("NA")  # En cualquier otro caso asignar 'NA'
)

# Mostrar el DataFrame con la nueva columna Job Type
display(df_transformed.select("Job Title", "Job Type"))


37. Muestra los registros únicos del campo `Job Type`. 

In [0]:
# Obtener los registros únicos del campo Job Type
unique_job_types = df_transformed.select("Job Type").distinct()

# Contar el número de registros únicos
total_unique = unique_job_types.count()
print(f"Total de valores únicos en 'Job Type': {total_unique}")

# Mostrar los valores únicos
display(unique_job_types)


38. Partiendo del campo `Job Description` se extraer todas o las principales skills solicitadas por las empresas, por ejemplo: Python, Spark , Big Data. Cada Skill debe ir en una nueva columna de tipo Binaria ( 0 , 1) o Booleana (True,  False) de modo que cada skill va ser una nueva columna y si esa skill es solicitada por la empresa colocar 1 sino colocar 0. Por ejemplo:  

In [0]:
from pyspark.sql.functions import when, col

# Definir las skills a buscar en las descripciones de trabajo
skills = ["Python", "Spark", "Big Data", "Excel", "Hadoop", "Machine Learning", "SQL", "R", "Data Science", "Deep Learning"]

# Crear nuevas columnas para cada skill
df_transformed = df
for skill in skills:
    df_transformed = df_transformed.withColumn(
        skill, 
        when(col("Job Description").rlike(f"(?i){skill}"), 1).otherwise(0)  # 1 si la skill está presente, 0 si no lo está
    )

# Mostrar el DataFrame con las nuevas columnas de skills
display(df_transformed.select("Job Title", *skills))


Por ejemplo:  
| Job Title         | Salary Estimate | Job Description                                 | Rating | Company Name       | Size       | Founded | Type of ownership         | Industry                       | Sector                         | Same State      | company_age | Python | Excel |
|--------------------|-----------------|-------------------------------------------------|--------|--------------------|------------|---------|---------------------------|--------------------------------|--------------------------------|----------------|-------------|--------|-------|
| Sr Data Scientist | 137000-171000   | Description The Senior Data Scientist is resp... | 3.1    | Healthfirst        | 1001-5000  | 1993    | Nonprofit Organization    | Insurance Carriers            | Insurance Carriers            | Same State      | 31          | 0      | 0     |
| Data Scientist    | 137000-171000   | Secure our Nation, Ignite your Future Join th... | 4.2    | ManTech            | 5001-10000 | 1968    | Public Company            | Research and Development      | Research and Development      | Same State      | 56          | 0      | 0     |
| Data Scientist    | 137000-171000   | Overview Analysis Group is one of the larges... | 3.8    | Analysis Group      | 1001-5000  | 1981    | Private Company           | Consulting                    | Consulting                    | Same State      | 43          | 1      | 1     |
| Data Scientist    | 137000-171000   | JOB DESCRIPTION: Do you have a passion for Da... | 3.5    | INFICON            | 501-1000   | 2000    | Public Company            | Electrical and Electronic Manufacturing | Electrical and Electronic Manufacturing | Different State | 24          | 1      | 1     |


In [0]:
from pyspark.sql.functions import when, col

# Definir las skills a buscar en las descripciones de trabajo
skills = ["Python", "Spark", "Big Data", "Excel", "Hadoop", "Machine Learning", "SQL", "R", "Data Science", "Deep Learning"]

# Crear nuevas columnas para cada skill
df_transformed = df
for skill in skills:
    df_transformed = df_transformed.withColumn(
        skill, 
        when(col("Job Description").rlike(f"(?i){skill}"), 1).otherwise(0)  # 1 si la skill está presente, 0 si no lo está
    )

# Mostrar el DataFrame con las nuevas columnas de skills
display(df_transformed.select("Job Title", *skills))


39. Exportar dataset final a csv

In [0]:
display(df)

40. Extraer todos los insights posibles que sean de valor o utilidad. Cree nuevas columnas, agrupar,  filtrar hacer varios plots que muestren dichos insights que sean de utilidad para una empresa o para un usuario. Elabore conclusiones con los insights encontrados. 